In [9]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import os

# Read all CSV files
folder_path = "/Users/zhanyangliu/Desktop/income_data"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

dfs = []  # List to store DataFrames

for file in csv_files:
    try:
        # Detect encoding first
        encoding = "utf-8"  # Assume UTF-8, change if needed
        df = pd.read_csv(os.path.join(folder_path, file), encoding=encoding, low_memory=False)
        
        # Ensure 'STATE' column exists (may vary per dataset)
        if "STATE" in df.columns:
            dfs.append(df)
    
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Combine all CSVs into one DataFrame
df_csv = pd.concat(dfs, ignore_index=True)

# Aggregate data by STATE
df_csv_grouped = df_csv.groupby("STATE").agg({
    "N1": "sum",  # Total population per state
    "agi_stub": "mean",  # Example: Average AGI per state
}).reset_index()

# Rename columns
df_csv_grouped.rename(columns={"N1": "Total_Population", "agi_stub": "Avg_AGI"}, inplace=True)
# Rename state abbreviations to full state names
state_abbrev_to_name = {
    "AK": "Alaska", "AL": "Alabama", "AR": "Arkansas", "AZ": "Arizona", "CA": "California",
    "CO": "Colorado", "CT": "Connecticut", "DC": "District of Columbia", "DE": "Delaware",
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "IA": "Iowa", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana",
    "MA": "Massachusetts", "MD": "Maryland", "ME": "Maine", "MI": "Michigan", "MN": "Minnesota",
    "MO": "Missouri", "MS": "Mississippi", "MT": "Montana", "NC": "North Carolina", "ND": "North Dakota",
    "NE": "Nebraska", "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NV": "Nevada",
    "NY": "New York", "OH": "Ohio", "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania",
    "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas",
    "UT": "Utah", "VA": "Virginia", "VT": "Vermont", "WA": "Washington", "WI": "Wisconsin",
    "WV": "West Virginia", "WY": "Wyoming"
}

df_csv_grouped["STATE"] = df_csv_grouped["STATE"].map(state_abbrev_to_name)
print(df_csv_grouped.head())  # Check grouped data


Error reading 21incyallagi.csv: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte
Error reading 22incyallagi.csv: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte
        STATE  Total_Population  Avg_AGI
0      Alaska         7788980.0      4.5
1     Alabama        47258650.0      4.5
2    Arkansas        28459530.0      4.5
3     Arizona        72485590.0      4.5
4  California       412840200.0      4.5


/var/folders/1v/fl4d4bns3ljc9dhyl3068_c80000gn/T/ipykernel_57161/2409417585.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_csv = pd.concat(dfs, ignore_index=True)


In [14]:
df_xlsx = pd.read_excel("/Users/zhanyangliu/Desktop/income_data/sales_2018-2023.xlsx")

# Ensure state column matches in both datasets
df_xlsx.rename(columns={"State": "STATE"}, inplace=True)

print(df_xlsx.head())  # Check XLSX data


        STATE  Electric (EV)  Plug-In Hybrid Electric (PHEV)  \
0     Alabama           1300                            1500   
1      Alaska            500                             300   
2     Arizona          12600                            7700   
3    Arkansas            600                             800   
4  California         273500                          215000   

   Hybrid Electric (HEV)  Biodiesel  Ethanol/Flex (E85)  \
0                  31800          0              509500   
1                   5400          0               62000   
2                 102000          0              533600   
3                  20300          0              370600   
4                1085300          0             1615200   

   Compressed Natural Gas (CNG)  Propane  Hydrogen  Methanol  Gasoline  \
0                         17000        0         0         0   3733700   
1                          3700        0         0         0    484900   
2                         15500       

In [15]:
merged_df = pd.merge(df_csv_grouped, df_xlsx, on="STATE", how="inner")

# Save the merged file
merged_df.to_csv("merged_income_data.csv", index=False)

print("CSV and XLSX data successfully merged!")
print(merged_df.head())

CSV and XLSX data successfully merged!
    STATE  Total_Population  Avg_AGI  Electric (EV)  \
0  Alaska         7788980.0      4.5            500   
1  Alaska         7788980.0      4.5            700   
2  Alaska         7788980.0      4.5            900   
3  Alaska         7788980.0      4.5           1300   
4  Alaska         7788980.0      4.5           2700   

   Plug-In Hybrid Electric (PHEV)  Hybrid Electric (HEV)  Biodiesel  \
0                             300                   5400          0   
1                             400                   5800          0   
2                             400                   6200          0   
3                             500                   7300       7600   
4                             900                  10600       9600   

   Ethanol/Flex (E85)  Compressed Natural Gas (CNG)  Propane  Hydrogen  \
0               62000                          3700        0         0   
1               64000                          3200    